This notebook demonstrates how to use OptKeras, wrapper of Keras and Optuna

##  Set up Google Colab environment

To run in Google Colab, specify a directory in Google Drive. (GPU is recommended.)

To run in an environment other than Google Colab, just skip this code.

In [1]:
try:
    # Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')
    import os
    # Specify a directory in Google Drive
    dir = '/content/drive/My Drive/Colab Notebooks/OptKeras'
    os.chdir(dir)    
    # Check the environment info
    print('## Current working directory: ', os.getcwd())
    print('## Check the uptime. (Google Colab reboots every 12 hours)')
    !cat /proc/uptime | awk '{print "Uptime is " $1 /60 /60 " hours (" $1 " sec)"}'
    print('## Check the GPU info')
    !nvidia-smi
    print('## Check the OS') 
    !cat /etc/issue
    print('## Check the Python version') 
    !python --version
    print('## Check the memory')
    !free -h
    print('## Check the disk')
    !df -h
except:
    print('Run the code assuming the environment is not Google Colab.')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
## Current working directory:  /content/drive/My Drive/Colab Notebooks/OptKeras
## Check the uptime. (Google Colab reboots every 12 hours)
Uptime is 0.387753 hours (1395.91 sec)
## Check the GPU info
Wed Feb 13 10:14:02 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.79       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |

## Install Optuna 0.7.0

In [2]:
!pip install optuna==0.7.0

## Install the latest version of OptKeras

In [3]:
!pip install git+https://github.com/Minyus/optkeras.git

  Cloning https://github.com/Minyus/optkeras.git to /tmp/pip-req-build-w1xl0n06
  Stored in directory: /tmp/pip-ephem-wheel-cache-jyk4qiuj/wheels/ad/c8/9d/67471f9fd7c5fd15961a2e7aa3d15215212d0631ea96df8b5c
Successfully built optkeras


## Import modules

In [4]:
# import Keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Activation, Flatten, Dense, Conv2D
from keras.layers import MaxPooling2D, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, Adagrad, RMSprop, Adam, Adadelta, Adamax, Nadam
import keras.backend as K

import keras
print('Keras', keras.__version__)

import tensorflow as tf
print('TensorFlow', tf.__version__)

import numpy as np
import pandas as pd

# import Optuna and OptKeras after Keras
import optuna 
print('Optuna', optuna.__version__)

from optkeras.optkeras import OptKeras
import optkeras
print('OptKeras', optkeras.__version__)

Using TensorFlow backend.


Keras 2.2.4
TensorFlow 1.13.0-rc1
Optuna 0.7.0
OptKeras 0.0.1


## Set up Dataset

In [0]:
dataset_name = 'MNIST'

if dataset_name in ['MNIST', 'MNIST_1000samples']:
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    
    img_x, img_y = x_train.shape[1], x_train.shape[2]
    x_train = x_train.reshape(-1, img_x, img_y, 1)
    x_test = x_test.reshape(-1, img_x, img_y, 1)   
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    num_classes = 10
    input_shape = (img_x, img_y, 1)

In [6]:
print('x_train: ', x_train.shape)
print('y_train', y_train.shape)
print('x_test: ', x_test.shape)
print('y_test', y_test.shape)
print('input_shape: ', input_shape )    

x_train:  (60000, 28, 28, 1)
y_train (60000,)
x_test:  (10000, 28, 28, 1)
y_test (10000,)
input_shape:  (28, 28, 1)


## Try a simple Convolutional Neural Networks model

In [7]:
model = Sequential()
model.add(Conv2D(128, kernel_size = (3, 3), strides = (1, 1),
               activation = 'relu', input_shape = input_shape))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))
model.compile(optimizer = Adam(), 
            loss='sparse_categorical_crossentropy', metrics=['accuracy'])			  
model.fit(x_train, y_train, validation_data = (x_test, y_test), shuffle = True,
          batch_size = 512, epochs = 2) 

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 8s 127us/step - loss: 0.3313 - acc: 0.9079 - val_loss: 0.1339 - val_acc: 0.9661
Epoch 2/2
60000/60000 [==============================] - 6s 95us/step - loss: 0.0986 - acc: 0.9733 - val_loss: 0.0767 - val_acc: 0.9767


## Setup optimization of a Keras model

In [8]:
study_name = dataset_name + '_Optimized'

ok = OptKeras( 
    # parameters for optuna.create_study
    storage='sqlite:///' + study_name + '_Optuna.db', 
    sampler=None, 
    pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=0), 
    study_name = study_name, 
    direction='minimize', 
    load_if_exists = True,
    # parameters for OptKeras
    monitor='val_error', # Either 'val_error' (1 - val_acc) or 'val_loss'
    enable_pruning=False, 
    models_to_keep=1, # Either 1, 0, or -1 (save all models) 
    verbose=1 )

def objective(trial): 
    epochs = 10
    
    K.clear_session()   
    model = Sequential()
    
    if trial.suggest_int('Conv', 0, 1):  
        # 1 Convolution layer
        i = 1
        model.add(Conv2D(
            filters = int(trial.suggest_discrete_uniform(
                'Conv_{}_num_filters_'.format(i), 32, 64, 32)), 
            kernel_size=tuple([trial.suggest_int(
                'Conv_{}_kernel_size'.format(i), 2, 3)] * 2),
            activation='relu',
            input_shape = input_shape))
        model.add(MaxPooling2D(pool_size=tuple([trial.suggest_int(
                'Conv_{}_max_pooling_size'.format(i), 2, 3)] * 2)))
        model.add(Dropout(trial.suggest_discrete_uniform(
                'Conv_{}_dropout_rate'.format(i), 0, 0.5, 0.25) ))
        model.add(Flatten())        
    else:
        model.add(Flatten(input_shape=input_shape))
    # 2 Fully connected layers
    for i in np.arange(2) + 1:
        model.add(Dense(int(trial.suggest_discrete_uniform(
            'FC_{}_num_hidden_units'.format(i), 256, 512, 256))))
        if trial.suggest_int('FC_{}_batch_normalization'.format(i), 0, 1):
            model.add(BatchNormalization())
        model.add(Activation(trial.suggest_categorical(
            'FC_{}_acivation'.format(i), ['relu'])))
        model.add(Dropout(
            trial.suggest_discrete_uniform(
                'FC_{}_dropout_rate'.format(i), 0, 0.5, 0.25) ))
        
    # Output layer    
    model.add(Dense(num_classes, activation='softmax'))
    
    optimizer_dict = { \
    #'Adagrad': Adagrad(),
    'Adam': Adam() }
    
    model.compile(optimizer = optimizer_dict[
        trial.suggest_categorical('Optimizer', list(optimizer_dict.keys()))],
          loss='sparse_categorical_crossentropy', metrics=['accuracy'])    
    
    if ok.verbose >= 2: model.summary()
    
    batch_size = trial.suggest_int('Batch_size', 256, 256) 
    #batch_size = int(trial.suggest_discrete_uniform(
    #                  'Batch_size', 256, 512, 256) )
    data_augmentation = trial.suggest_int('Data_augmentation', 0, 1)
    
    if not data_augmentation:
        # [Required] Specify callbacks(trial) in fit method
        model.fit(x_train, y_train, batch_size = batch_size,
                  epochs = epochs, validation_data = (x_test, y_test),
                  shuffle = True,
                  callbacks = ok.callbacks(trial), 
                  verbose = ok.keras_verbose )
    
    if data_augmentation:
        # This will do preprocessing and realtime data augmentation:
        datagen = ImageDataGenerator(
            width_shift_range=[-1, 0, +1], # 1 pixel
            height_shift_range=[-1, 0, +1], # 1 pixel
            zoom_range=[0.95,1.05],  # set range for random zoom
            horizontal_flip=False,  # disable horizontal flip
            vertical_flip=False )  # disable vertical flip
        datagen.fit(x_train)
        # [Required] Specify callbacks(trial) in fit_generator method
        model.fit_generator(datagen.flow(x_train, y_train, 
                                         batch_size=batch_size),
                            epochs=epochs, validation_data=(x_test, y_test),
                            steps_per_epoch=len(x_train) // batch_size,
                            callbacks = ok.callbacks(trial), 
                            verbose = ok.keras_verbose )  
    
    # [Required] return trial_best_value (recommended) or latest_value
    return ok.trial_best_value

[I 2019-02-13 10:14:40,446] A new study created with name: MNIST_Optimized


[2019-02-13 10:14:40.478006]  Ready for optimization. (message printed as verbose is set to 1+)


## Run optimization trials

In [9]:
# Set n_trials and/or timeout (in sec) for optimization by Optuna
ok.optimize(objective, n_trials=10, timeout = 12 * 60 * 60)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
[2019-02-13 10:14:41.643727] Latest trial id: None, value: None (None) | Best trial id: None, value: None, parameters: None


[I 2019-02-13 10:17:23,849] Finished a trial resulted in value: 0.040200000000000014. Current best value is 0.040200000000000014 with parameters: {'Batch_size': 256, 'Conv': 0, 'Data_augmentation': 1, 'FC_1_acivation': 'relu', 'FC_1_batch_normalization': 1, 'FC_1_dropout_rate': 0.5, 'FC_1_num_hidden_units': 512.0, 'FC_2_acivation': 'relu', 'FC_2_batch_normalization': 1, 'FC_2_dropout_rate': 0.25, 'FC_2_num_hidden_units': 512.0, 'Optimizer': 'Adam'}.


[2019-02-13 10:17:24.710872] Latest trial id: 1, value: 0.040200000000000014 (TrialState.COMPLETE) | Best trial id: 1, value: 0.040200000000000014, parameters: {'Batch_size': 256, 'Conv': 0, 'Data_augmentation': 1, 'FC_1_acivation': 'relu', 'FC_1_batch_normalization': 1, 'FC_1_dropout_rate': 0.5, 'FC_1_num_hidden_units': 512.0, 'FC_2_acivation': 'relu', 'FC_2_batch_normalization': 1, 'FC_2_dropout_rate': 0.25, 'FC_2_num_hidden_units': 512.0, 'Optimizer': 'Adam'}


[I 2019-02-13 10:20:04,556] Finished a trial resulted in value: 0.03510000000000002. Current best value is 0.03510000000000002 with parameters: {'Batch_size': 256, 'Conv': 0, 'Data_augmentation': 1, 'FC_1_acivation': 'relu', 'FC_1_batch_normalization': 1, 'FC_1_dropout_rate': 0.0, 'FC_1_num_hidden_units': 256.0, 'FC_2_acivation': 'relu', 'FC_2_batch_normalization': 0, 'FC_2_dropout_rate': 0.0, 'FC_2_num_hidden_units': 512.0, 'Optimizer': 'Adam'}.


[2019-02-13 10:20:05.520462] Latest trial id: 2, value: 0.03510000000000002 (TrialState.COMPLETE) | Best trial id: 2, value: 0.03510000000000002, parameters: {'Batch_size': 256, 'Conv': 0, 'Data_augmentation': 1, 'FC_1_acivation': 'relu', 'FC_1_batch_normalization': 1, 'FC_1_dropout_rate': 0.0, 'FC_1_num_hidden_units': 256.0, 'FC_2_acivation': 'relu', 'FC_2_batch_normalization': 0, 'FC_2_dropout_rate': 0.0, 'FC_2_num_hidden_units': 512.0, 'Optimizer': 'Adam'}


...

[2019-02-13 10:25:35.909683] Latest trial id: 8, value: 0.017800000000000038 (TrialState.COMPLETE) | Best trial id: 4, value: 0.009199999999999986, parameters: {'Batch_size': 256, 'Conv': 1, 'Conv_1_dropout_rate': 0.25, 'Conv_1_kernel_size': 3, 'Conv_1_max_pooling_size': 3, 'Conv_1_num_filters_': 64.0, 'Data_augmentation': 0, 'FC_1_acivation': 'relu', 'FC_1_batch_normalization': 0, 'FC_1_dropout_rate': 0.5, 'FC_1_num_hidden_units': 256.0, 'FC_2_acivation': 'relu', 'FC_2_batch_normalization': 1, 'FC_2_dropout_rate': 0.25, 'FC_2_num_hidden_units': 256.0, 'Optimizer': 'Adam'}


[I 2019-02-13 10:25:58,504] Finished a trial resulted in value: 0.02429999999999999. Current best value is 0.009199999999999986 with parameters: {'Batch_size': 256, 'Conv': 1, 'Conv_1_dropout_rate': 0.25, 'Conv_1_kernel_size': 3, 'Conv_1_max_pooling_size': 3, 'Conv_1_num_filters_': 64.0, 'Data_augmentation': 0, 'FC_1_acivation': 'relu', 'FC_1_batch_normalization': 0, 'FC_1_dropout_rate': 0.5, 'FC_1_num_hidden_units': 256.0, 'FC_2_acivation': 'relu', 'FC_2_batch_normalization': 1, 'FC_2_dropout_rate': 0.25, 'FC_2_num_hidden_units': 256.0, 'Optimizer': 'Adam'}.


[2019-02-13 10:26:00.252350] Latest trial id: 9, value: 0.02429999999999999 (TrialState.COMPLETE) | Best trial id: 4, value: 0.009199999999999986, parameters: {'Batch_size': 256, 'Conv': 1, 'Conv_1_dropout_rate': 0.25, 'Conv_1_kernel_size': 3, 'Conv_1_max_pooling_size': 3, 'Conv_1_num_filters_': 64.0, 'Data_augmentation': 0, 'FC_1_acivation': 'relu', 'FC_1_batch_normalization': 0, 'FC_1_dropout_rate': 0.5, 'FC_1_num_hidden_units': 256.0, 'FC_2_acivation': 'relu', 'FC_2_batch_normalization': 1, 'FC_2_dropout_rate': 0.25, 'FC_2_num_hidden_units': 256.0, 'Optimizer': 'Adam'}


[I 2019-02-13 10:28:51,146] Finished a trial resulted in value: 0.02059999999999995. Current best value is 0.009199999999999986 with parameters: {'Batch_size': 256, 'Conv': 1, 'Conv_1_dropout_rate': 0.25, 'Conv_1_kernel_size': 3, 'Conv_1_max_pooling_size': 3, 'Conv_1_num_filters_': 64.0, 'Data_augmentation': 0, 'FC_1_acivation': 'relu', 'FC_1_batch_normalization': 0, 'FC_1_dropout_rate': 0.5, 'FC_1_num_hidden_units': 256.0, 'FC_2_acivation': 'relu', 'FC_2_batch_normalization': 1, 'FC_2_dropout_rate': 0.25, 'FC_2_num_hidden_units': 256.0, 'Optimizer': 'Adam'}.


[2019-02-13 10:28:51.254180] Latest trial id: 10, value: 0.02059999999999995 (TrialState.COMPLETE) | Best trial id: 4, value: 0.009199999999999986, parameters: {'Batch_size': 256, 'Conv': 1, 'Conv_1_dropout_rate': 0.25, 'Conv_1_kernel_size': 3, 'Conv_1_max_pooling_size': 3, 'Conv_1_num_filters_': 64.0, 'Data_augmentation': 0, 'FC_1_acivation': 'relu', 'FC_1_batch_normalization': 0, 'FC_1_dropout_rate': 0.5, 'FC_1_num_hidden_units': 256.0, 'FC_2_acivation': 'relu', 'FC_2_batch_normalization': 1, 'FC_2_dropout_rate': 0.25, 'FC_2_num_hidden_units': 256.0, 'Optimizer': 'Adam'}


## Check the results

In [10]:
# OptKeras best_trial returns 
print('Best trial id: ', ok.best_trial.trial_id)
print('Best value:', ok.best_trial.value)
print('Best params: ')
ok.best_trial.params

Best trial id:  4
Best value: 0.009199999999999986
Best params: 


{'Batch_size': 256,
 'Conv': 1,
 'Conv_1_dropout_rate': 0.25,
 'Conv_1_kernel_size': 3,
 'Conv_1_max_pooling_size': 3,
 'Conv_1_num_filters_': 64.0,
 'Data_augmentation': 0,
 'FC_1_acivation': 'relu',
 'FC_1_batch_normalization': 0,
 'FC_1_dropout_rate': 0.5,
 'FC_1_num_hidden_units': 256.0,
 'FC_2_acivation': 'relu',
 'FC_2_batch_normalization': 1,
 'FC_2_dropout_rate': 0.25,
 'FC_2_num_hidden_units': 256.0,
 'Optimizer': 'Adam'}

In [11]:
"""
Alternatively, you can access Optuna's study object to, for example, 
get the best parameters as well.
Please note that study.best_trial returns error if optimization trials 
were not completed (e.g. if you interupt execution) as of Optuna 0.7.0, 
so usage of OptKeras is recommended.
"""
study = ok.study
study.best_trial.params 

{'Batch_size': 256,
 'Conv': 1,
 'Conv_1_dropout_rate': 0.25,
 'Conv_1_kernel_size': 3,
 'Conv_1_max_pooling_size': 3,
 'Conv_1_num_filters_': 64.0,
 'Data_augmentation': 0,
 'FC_1_acivation': 'relu',
 'FC_1_batch_normalization': 0,
 'FC_1_dropout_rate': 0.5,
 'FC_1_num_hidden_units': 256.0,
 'FC_2_acivation': 'relu',
 'FC_2_batch_normalization': 1,
 'FC_2_dropout_rate': 0.25,
 'FC_2_num_hidden_units': 256.0,
 'Optimizer': 'Adam'}

In [12]:
## Check the Optuna CSV log file 
pd.options.display.max_rows = 8 # limit rows to display
print('Data Frame read from', ok.optuna_log_file_path, '\n')
pd.read_csv(ok.optuna_log_file_path)

Data Frame read from MNIST_Optimized_Optuna.csv 



,trial_id,state,value,datetime_start,datetime_complete,Batch_size,Conv,Conv_1_dropout_rate,Conv_1_kernel_size,Conv_1_max_pooling_size,...,_Datetime_epoch_begin,_Datetime_epoch_end,_Monitor,_Trial_id,acc,error,loss,val_acc,val_error,val_loss
0,1,TrialState.COMPLETE,0.0402,2019-02-13 10:14:40.506402,2019-02-13 10:17:23.806408,256,0,NaN,NaN,NaN,...,2019-02-13 10:14:42.603022,2019-02-13 10:14:59.145381,val_error,1,0.966859,0.138989,0.108857,0.9849,0.0402,0.044857
1,2,TrialState.COMPLETE,0.0351,2019-02-13 10:17:23.859359,2019-02-13 10:20:04.501119,256,0,NaN,NaN,NaN,...,2019-02-13 10:17:25.409510,2019-02-13 10:17:41.593029,val_error,2,0.985170,0.093333,0.047310,0.9846,0.0351,0.048114
2,3,TrialState.COMPLETE,0.0177,2019-02-13 10:20:04.569554,2019-02-13 10:20:33.211843,256,0,NaN,NaN,NaN,...,2019-02-13 10:20:30.641364,2019-02-13 10:20:32.918594,val_error,3,0.984500,0.015500,0.046144,0.9823,0.0177,0.058114
3,4,TrialState.COMPLETE,0.0092,2019-02-13 10:20:33.265991,2019-02-13 10:21:18.569915,256,1,0.25,3.0,3.0,...,2019-02-13 10:21:14.369394,2019-02-13 10:21:18.143973,val_error,4,0.990067,0.009933,0.031785,0.9908,0.0092,0.030390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,7,TrialState.COMPLETE,0.0188,2019-02-13 10:24:44.198139,2019-02-13 10:25:09.546023,256,0,NaN,NaN,NaN,...,2019-02-13 10:25:07.235147,2019-02-13 10:25:09.181546,val_error,7,0.992600,0.007400,0.022032,0.9812,0.0188,0.063693
7,8,TrialState.COMPLETE,0.0178,2019-02-13 10:25:09.612843,2019-02-13 10:25:34.816422,256,0,NaN,NaN,NaN,...,2019-02-13 10:25:32.469296,2019-02-13 10:25:34.366723,val_error,8,0.996067,0.003933,0.012279,0.9822,0.0178,0.063992
8,9,TrialState.COMPLETE,0.0243,2019-02-13 10:25:34.882716,2019-02-13 10:25:58.442689,256,0,NaN,NaN,NaN,...,2019-02-13 10:25:56.290366,2019-02-13 10:25:58.073696,val_error,9,0.997567,0.002433,0.008477,0.9757,0.0243,0.092641
9,10,TrialState.COMPLETE,0.0206,2019-02-13 10:25:58.514614,2019-02-13 10:28:51.081418,256,1,0.00,3.0,3.0,...,2019-02-13 10:26:01.156709,2019-02-13 10:26:18.801302,val_error,10,0.984333,0.102147,0.051120,0.9870,0.0206,0.041684


In [13]:
## Check the Keras CSV log file

pd.options.display.max_rows = 8 # limit rows to display
print('Data Frame read from', ok.keras_log_file_path, '\n')
pd.read_csv(ok.keras_log_file_path)

Data Frame read from MNIST_Optimized_Keras.csv 



,epoch,_Datetime_epoch_begin,_Datetime_epoch_end,_Monitor,_Trial_id,acc,error,loss,val_acc,val_error,val_loss
0,0,2019-02-13 10:14:42.603022,2019-02-13 10:14:59.145381,val_error,1,0.861011,0.138989,0.443171,0.9598,0.0402,0.125782
1,1,2019-02-13 10:14:42.603022,2019-02-13 10:14:59.145381,val_error,1,0.930604,0.138989,0.225478,0.9728,0.0402,0.087606
2,2,2019-02-13 10:14:42.603022,2019-02-13 10:14:59.145381,val_error,1,0.943978,0.138989,0.180567,0.9778,0.0402,0.071474
3,3,2019-02-13 10:14:42.603022,2019-02-13 10:14:59.145381,val_error,1,0.951727,0.138989,0.156699,0.9788,0.0402,0.061810
...,...,...,...,...,...,...,...,...,...,...,...
96,6,2019-02-13 10:26:01.156709,2019-02-13 10:26:18.801302,val_error,10,0.981253,0.102147,0.061509,0.9918,0.0206,0.025047
97,7,2019-02-13 10:26:01.156709,2019-02-13 10:26:18.801302,val_error,10,0.981253,0.102147,0.059762,0.9906,0.0206,0.029333
98,8,2019-02-13 10:26:01.156709,2019-02-13 10:26:18.801302,val_error,10,0.981689,0.102147,0.058474,0.9902,0.0206,0.029613
99,9,2019-02-13 10:26:01.156709,2019-02-13 10:26:18.801302,val_error,10,0.984333,0.102147,0.051120,0.9870,0.0206,0.041684
